In [8]:
#pip install widgetsnbextension

In [7]:
#pip install ipywidgets

In [17]:
#pip install voila

In [5]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [ ]:

%%capture
pip install widgetsnbextension
pip install ipywidgets
pip install voila
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix

In [71]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
import os

In [72]:
checkpoint = widgets.ToggleButtons(
            options=['lookingglass_dalle_12000.pt','lookingglass_dalle_2000.pt','lookingglass_dalle_4000.pt',
                     'lookingglass_dalle_6000.pt','lookingglass_dalle_8000.pt','lookingglass_dalle_10000.pt',
                     'lookingglass_dalle_12000.pt','lookingglass_dalle_14000.pt','lookingglass_dalle_16000.pt',
                    'lookingglass_dalle_18000.pt','lookingglass_dalle_20000.pt']
        )

In [73]:
checkpoint = widgets.ToggleButtons(
            options=[
                    'lookingglass_dalle_4000.pt']
        )

In [ ]:
img_amount = widgets.IntSlider(value=2,min=1,max=9,step=1,description='Image amount')
filtered = widgets.IntSlider(value=2,min=1,max=9,step=1,description='Filtered Images')

In [ ]:
display(img_amount)
display(filtered)

In [74]:
confidence = widgets.ToggleButtons(
            options=['Ultra-Low', 'Low', 'Medium', 'High', 'Ultra-High'],description='confidence')
variability = widgets.ToggleButtons(
            options=['Ultra-Low', 'Low', 'Medium', 'High', 'Ultra-High'], description = 'variability')

In [75]:
prompt = widgets.Text(placeholder='Input your prompt')

In [76]:
display(confidence)

ToggleButtons(description='confidence', options=('Ultra-Low', 'Low', 'Medium', 'High', 'Ultra-High'), value='U…

In [77]:
display(variability)

ToggleButtons(description='variability', options=('Ultra-Low', 'Low', 'Medium', 'High', 'Ultra-High'), value='…

In [78]:
display(prompt)

Text(value='', placeholder='Input your prompt')

In [79]:
button_run = widgets.Button(description = 'Run model', tooltip='Send',
                style={'description_width': 'initial'})

In [80]:
output = widgets.Output()

In [81]:
from rudalle import get_rudalle_model, get_tokenizer, get_vae
import torch
from model.functions import generate

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [83]:
if device.type == "cpu":
    vae = get_vae().to("cpu")
    model = get_rudalle_model("Malevich", pretrained=True, fp16=False, device=device)
else:
    vae = get_vae().to("cuda:0")
    model = get_rudalle_model("Malevich", pretrained=True, fp16=True, device=device)

Working with z of shape (1, 256, 32, 32) = 262144 dimensions.
vae --> ready
◼️ Malevich is 1.3 billion params model from the family GPT3-like, that uses Russian language and text+image multi-modality.


In [99]:
import os

def on_button_clicked(event):
    with output:
        clear_output()
        model_path = checkpoint.value
        filepath = f'{prompt.value}-{checkpoint.value}-{confidence.value}-{variability.value}'
        model.load_state_dict(torch.load(model_path, map_location ='cpu'))
        with torch.no_grad():

            generate(vae, model, prompt.value, confidence = confidence.value, variability = variability.value, rurealesrgan_multiplier="x1", output_filepath=filepath, num_filtered = 1, image_amount = 1)
        print(f'Images saved in {filepath}')

        for image in os.listdir(filepath):
            Image(f'filepath/{image}')


In [100]:
button_run.on_click(on_button_clicked)

In [101]:
vbox_result = widgets.VBox([button_run, output])

In [105]:
display(button_run, output)

Button(description='Run model', style=ButtonStyle(), tooltip='Send')

Output(outputs=({'output_type': 'error', 'ename': 'NoResult', 'evalue': 'No service has returned a valid resul…